In [27]:

import pandas as pd
import glob as glob
import os
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

def main():

    path = define_path()
    data = concat_files(path)
    data = transform_data(data)
    data = clean_data(data)
    bi_grams = bi_gram_data(data)
    skills = skills_data(data)
    save_df(data,skills,bi_grams)

def define_path():
    path= input('Please define path of your xlsx files with  * to add all xlsx you want to concat')
    return path

def concat_files(path):
    files=glob.glob(path)
    data=pd.concat(map(pd.read_excel,files),ignore_index=True)
    return data
    
def transform_data(data):
    data=data.drop_duplicates(['Name','Company'])
    data=data.drop('Column1',axis=1)
    sep=data['Contract'].str.split(' ',n=1,expand= True)
    data=data.drop('Contract',axis=1)
    data['Contract']=sep.iloc[:,0].to_list()
    data['Level']=sep.iloc[:,1].to_list()

    data['Level']=data['Level'].str.replace('· ','')

    sep=data['Industry'].str.split(' ',n=2,expand= True)
    data=data.drop('Industry',axis=1)

    data['Size ']=sep.iloc[:,0].to_list()
    data['Industry']=sep.iloc[:,2].to_list()
    data['Industry']=data['Industry'].str.replace('· ','')

    return data


def clean_data(data):
    from gensim.parsing.preprocessing import remove_stopwords
     ## Lower case

    for column in data.columns:
        
        data[column] = data[column].str.lower()
         
     ## remove tabulation and punctuation
    data['Description'] = data['Description'].str.replace('[^\w\s]',' ')
     ## digits
    #data['Description'] = data['Description'].str.replace('\d+', '')

     # Remove stopwords
    data['Description']=data['Description'].apply(lambda x: remove_stopwords(x))
    

     #Remove Serveral words
    data['Description'] = data['Description'].str.replace('costa rica', '')
    data['Description'] = data['Description'].str.replace('equal opportunity', '')
    data['Description'] = data['Description'].str.replace('sexual orientation', '')
    data['Description'] = data['Description'].str.replace('gender identity', '')
    data['Description'] = data['Description'].str.replace('team member', '')
    data['Description'] = data['Description'].str.replace('national origin', '')
    data['Description'] = data['Description'].str.replace('data analytics', 'data analysis')
    
    return data



def bi_gram_data(data, n=2):

    bag_words_data = pd.DataFrame(columns=['Bi-gram','Count','Data Postion'])

    Bi_gram = []
    Count = []
    Data_pos = []
     
    for position in data['Data Position'].unique().tolist():
        
        text=data[data['Data Position']== position]['Description']
    
        new= text.str.split()
        new=new.values.tolist()
        corpus=[word for i in new for word in i]

        def _get_top_ngram(corpus, n=None):
             vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
             bag_of_words = vec.transform(corpus)
             sum_words = bag_of_words.sum(axis=0) 
             words_freq = [(word, sum_words[0, idx]) 
                      for word, idx in vec.vocabulary_.items()]
             words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
             return words_freq

        top_n_bigrams=_get_top_ngram(text,n)
         
        for i in range(len(top_n_bigrams)):
            
            Bi_gram.append(top_n_bigrams[i][0])
            Count.append(top_n_bigrams[i][1])
            Data_pos.append(position)

    bag_words_data['Bi-gram'] = Bi_gram
          
    bag_words_data['Count'] = Count

    bag_words_data['Data Postion'] = Data_pos


    return bag_words_data





def skills_data(data):
    
    skills_df=pd.DataFrame(columns=['Name','Count','Data Position'])
    
    name=[]
    count_skill=[]
    data_pos=[]

    
    for position in data['Data Position'].unique().tolist():
        
         text=data[data['Data Position']== position]['Description']
    
         text= text.tolist()
         text=' '.join(text)

         skills= ['python','sql','scala','excel',' r ','julia','spark','cloud','azure','aws','databricks','html','knime','git hub','tableau','power bi',' dash ','shiny','google looker','hadoop',' sas ','matlab','version control','keras','pytorch','deep learning']

    

         for skill in skills:

             name.append(skill)

             count_skill.append(text.count(skill))

             data_pos.append(position)



    skills_df['Name'] = name
    skills_df['Count'] = count_skill
    skills_df['Data Position'] = data_pos

    skills_df=skills_df.sort_values('Count',ascending=False)

    return skills_df


def save_df(data,skills,bi_grams):
      # save the df
           skills.to_excel('skills_count.xlsx', engine='xlsxwriter')
           bi_grams.to_excel('bi_grams_count.xlsx',engine='xlsxwriter')
           data.to_excel('linkedin data jobs.xlsx',engine='xlsxwriter')


if __name__=='__main__':
    main()


/var/folders/7x/p6g2f1xs1qq5s4_nn_mvnygc0000gn/T/ipykernel_20684/3441374631.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Description'] = data['Description'].str.replace('[^\w\s]',' ')
